# Computer Vision for License Plate Recognition
The objective of this challenge is to make use of `object detection` and `image classification` to tell the values on the number plates (`Tunisian`) of a car from its image

## Approach
- create two models
- one to detect license plates
- another to detect and numbers in the license plates

In [289]:
## matrix and dataframe packages
import numpy as np
import pandas as pd

## image packages
import PIL as pil
import matplotlib.pyplot as plt

## deep learning packages
import torch
import torchvision.models as models
import torchvision.transforms as torch_transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

## utility packages
import os
from datetime import datetime

### Preparing paths

In [290]:
data_path = "x__data"

## images path
detection_imgs_path = f"{data_path}/license_plates_detection_train/license_plates_detection_train/"
recognition_imgs_path = f"{data_path}/license_plates_recognition_train/license_plates_recognition_train/"
test_path = f"{data_path}/text/text_private/"

## annotations path
detection_annots = f"{data_path}/license_plates_detection_train.csv"
recognition_annots = f"{data_path}/license_plates_recognition_train.csv"

### Checking whether all images are unique

In [291]:
pd.read_csv(detection_annots).nunique()

img_id    900
ymin      293
xmin      412
ymax      286
xmax      413
dtype: int64

### Creating Datasets

In [292]:
class DetectionImagesDataset(torch.utils.data.Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.annots = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform
        self.unique_imgs = self.annots["img_id"].unique()

    def __len__(self):
        return len(self.annots)

    def __getitem__(self, idx):
        img_name = self.unique_imgs[idx]
        image = pil.Image.open(f"{self.img_dir}/{img_name}").convert("RGB")
        boxes = self.annots[self.annots["img_id"] == img_name][["xmin","ymin","xmax","ymax"]].values.astype(np.float32)
        labels = torch.ones(1, dtype=torch.int64)
        target = {"boxes": torch.from_numpy(boxes),
                  "labels": labels
                 }

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            target = self.target_transform(target)

        return image, target
    
class RecognitionImagesDataset(torch.utils.data.Dataset):
    pass
    
class TestImagesDataset(torch.utils.data.Dataset):
    pass

### Instantiating Datasets

In [293]:
data_transform = torch_transforms.Compose([torch_transforms.Resize((224,224), interpolation=torch_transforms.InterpolationMode.BICUBIC),
                                           torch_transforms.ToTensor()
                                          ])

train_dataset = DetectionImagesDataset(detection_annots, detection_imgs_path, transform=data_transform)

In [294]:
train_dataset[32]

(tensor([[[0.0118, 0.0235, 0.0196,  ..., 0.0157, 0.0196, 0.0392],
          [0.0118, 0.0235, 0.0235,  ..., 0.0196, 0.0275, 0.0431],
          [0.0118, 0.0235, 0.0275,  ..., 0.0314, 0.0353, 0.0471],
          ...,
          [0.4157, 0.4471, 0.4902,  ..., 0.4706, 0.4549, 0.4549],
          [0.4275, 0.4667, 0.4941,  ..., 0.4588, 0.4510, 0.4549],
          [0.4549, 0.4824, 0.4824,  ..., 0.4510, 0.4431, 0.4549]],
 
         [[0.0118, 0.0235, 0.0196,  ..., 0.0157, 0.0196, 0.0392],
          [0.0118, 0.0235, 0.0235,  ..., 0.0196, 0.0275, 0.0431],
          [0.0118, 0.0235, 0.0275,  ..., 0.0314, 0.0353, 0.0471],
          ...,
          [0.3608, 0.3765, 0.4039,  ..., 0.3765, 0.3686, 0.3804],
          [0.3686, 0.3961, 0.4078,  ..., 0.3686, 0.3647, 0.3804],
          [0.3725, 0.4039, 0.3922,  ..., 0.3725, 0.3686, 0.3882]],
 
         [[0.0118, 0.0235, 0.0235,  ..., 0.0157, 0.0196, 0.0392],
          [0.0118, 0.0235, 0.0275,  ..., 0.0196, 0.0275, 0.0431],
          [0.0118, 0.0235, 0.0353,  ...,

### Splitting dataset into `train` and `validation`

In [295]:
generator = torch.Generator().manual_seed(42)
train_detection_dataset, validation_detection_dataset = torch.utils.data.random_split(train_dataset,[.7,.3], generator=generator)

In [296]:
len(train_detection_dataset)

630

In [297]:
len(validation_detection_dataset)

270

### Creating Data Loaders

In [298]:
def collate_fn(data):
    return data

In [299]:
train_detection_loader = torch.utils.data.DataLoader(train_detection_dataset,
                                           batch_size=16,
                                            collate_fn=collate_fn,
                                           shuffle=True
                                          )
validation_detection_loader = torch.utils.data.DataLoader(validation_detection_dataset,
                                           batch_size=16,
                                        collate_fn=collate_fn,
                                          )

In [300]:
# next(iter(train_detection_loader))

### Getting the pretrained model

In [301]:
fastercnn = fasterrcnn_resnet50_fpn(weights="DEFAULT")

#### Changing the scoring head

In [302]:
in_features = fastercnn.roi_heads.box_predictor.cls_score.in_features
num_classes=2
fastercnn.box_predictor = FastRCNNPredictor(in_channels=in_features, num_classes=num_classes)

In [303]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

### Instantiating optimizer

In [304]:
optimizer = torch.optim.Adam(fastercnn.parameters())

### Moving model to device

In [305]:
fastercnn.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

### Training Function

In [306]:
def batch_gd(model, optimizer, train_loader, validation_loader=None, epochs=8):
    train_losses = []
    for it in range(epochs):
        t0 = datetime.now()
        epoch_losses = []
        for item in train_loader:
            optimizer.zero_grad()
            imgs = []
            targets = []
            for img, target in item:
                targets.append({"boxes": target["boxes"].to(device), "labels": target["labels"].to(device)})
                imgs.append(img.to(device))

            loss_dict = model(imgs, targets)
            loss = sum(v for v in loss_dict.values())
            loss.backward()
            optimizer.step()
            epoch_losses.append(loss.item())

        train_losses.append(epoch_losses.mean())

        dt = datetime.now() - t0

        print(f"Epoch {it}/{epochs} Train Loss: {train_losses[it]}, duration: {dt}")

    return train_losses

In [307]:
train_losses = batch_gd(fastercnn, optimizer, train_detection_loader)

KeyboardInterrupt: 